In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("Corona.csv")

In [3]:
df.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,11-03-2020,TRUE,FALSE,TRUE,FALSE,FALSE,negative,None,None,Abroad
1,2,11-03-2020,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
2,3,11-03-2020,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
3,4,11-03-2020,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Abroad
4,5,11-03-2020,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Contact with confirmed


In [4]:
df.shape

(278848, 11)

In [5]:
df.nunique()

Ind_ID                 278848
Test_date                  51
Cough_symptoms              5
Fever                       5
Sore_throat                 5
Shortness_of_breath         5
Headache                    5
Corona                      3
Age_60_above                3
Sex                         3
Known_contact               3
dtype: int64

In [6]:
df.drop(["Ind_ID"], axis=1, inplace = True)
# df.drop(["Test_date"], axis=1, inplace = True)

In [7]:
df.isnull().sum()

Test_date              0
Cough_symptoms         0
Fever                  0
Sore_throat            0
Shortness_of_breath    0
Headache               0
Corona                 0
Age_60_above           0
Sex                    0
Known_contact          0
dtype: int64

In [8]:
df.drop(["Age_60_above"], axis=1, inplace = True)

In [9]:
for i in df.columns:
  print("\nUnique values in '{}' are :\n".format(i),df[i].unique())


Unique values in 'Test_date' are :
 ['11-03-2020' '12-03-2020' '13-03-2020' '14-03-2020' '15-03-2020'
 '16-03-2020' '17-03-2020' '18-03-2020' '19-03-2020' '20-03-2020'
 '21-03-2020' '22-03-2020' '23-03-2020' '24-03-2020' '25-03-2020'
 '26-03-2020' '27-03-2020' '28-03-2020' '29-03-2020' '30-03-2020'
 '31-03-2020' '01-04-2020' '02-04-2020' '03-04-2020' '04-04-2020'
 '05-04-2020' '06-04-2020' '07-04-2020' '08-04-2020' '09-04-2020'
 '10-04-2020' '11-04-2020' '12-04-2020' '13-04-2020' '14-04-2020'
 '15-04-2020' '16-04-2020' '17-04-2020' '18-04-2020' '19-04-2020'
 '20-04-2020' '21-04-2020' '22-04-2020' '23-04-2020' '24-04-2020'
 '25-04-2020' '26-04-2020' '27-04-2020' '28-04-2020' '29-04-2020'
 '30-04-2020']

Unique values in 'Cough_symptoms' are :
 ['TRUE' 'FALSE' 'None' False True]

Unique values in 'Fever' are :
 ['FALSE' 'TRUE' 'None' False True]

Unique values in 'Sore_throat' are :
 ['TRUE' 'FALSE' 'None' False True]

Unique values in 'Shortness_of_breath' are :
 ['FALSE' 'TRUE' 'None'

In [10]:
df.replace("None", np.nan, inplace=True)
df = df.replace("other", np.nan)

In [11]:
df.shape

(278848, 9)

In [12]:
symptoms = ["Cough_symptoms", "Fever", "Sore_throat", "Shortness_of_breath", "Headache"]
for i in symptoms:
  df[i] = df[i].replace("TRUE",1)
  df[i] = df[i].replace("FALSE",0)

In [13]:
df.head()

,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Sex,Known_contact
0,11-03-2020,1,0,1,0,0,negative,NaN,Abroad
1,11-03-2020,0,1,0,0,0,positive,NaN,Abroad
2,11-03-2020,0,1,0,0,0,positive,NaN,Abroad
3,11-03-2020,1,0,0,0,0,negative,NaN,Abroad
4,11-03-2020,1,0,0,0,0,negative,NaN,Contact with confirmed


In [14]:
df["Corona"] = df["Corona"].replace("positive",1)
df["Corona"] = df["Corona"].replace("negative",0)

In [15]:
df["Sex"] = df["Sex"].replace("male",1)
df["Sex"] = df["Sex"].replace("female",0)

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
work_reshaped = np.array(df['Known_contact']).reshape(-1, 1)
xx = np.array(ct.fit_transform(work_reshaped))
print(xx)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [17]:
work_reshape = pd.DataFrame(xx, columns=['Abroad', 'Contact with confirmed', 'Other'])
df1 = pd.concat([work_reshape, df], axis = 1)

In [18]:
df1.drop(["Known_contact"], axis=1, inplace = True)
df1.head()

,Abroad,Contact with confirmed,Other,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Sex
0,1.0,0.0,0.0,11-03-2020,1,0,1,0,0,0.0,NaN
1,1.0,0.0,0.0,11-03-2020,0,1,0,0,0,1.0,NaN
2,1.0,0.0,0.0,11-03-2020,0,1,0,0,0,1.0,NaN
3,1.0,0.0,0.0,11-03-2020,1,0,0,0,0,0.0,NaN
4,0.0,1.0,0.0,11-03-2020,1,0,0,0,0,0.0,NaN


In [19]:
df1.dropna(inplace = True)

In [20]:
df1.shape

(255668, 11)

In [21]:
X = df1.drop(["Corona"], axis=1)
y = df1["Corona"]

In [22]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

X = preprocessor.fit_transform(X)

In [23]:
df1['Test_date'] = pd.to_datetime(df1['Test_date'], format='%d-%m-%Y')

In [24]:
before_df = df1.loc[df1["Test_date"]>"15-04-2020"].shape[0]
after_df = df1.loc[df1["Test_date"]>"15-04-2020"].shape[0]

In [25]:
split_size = before_df/(before_df+after_df)
split_size

0.5

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state=0)

In [27]:
X_train.shape, y_train.shape

((127834, 54), (127834,))

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)
print("Accuracy of Logistic Regression Model : {}%".format(round(accuracy_score(y_pred, y_test)*100)))
print("Classification Report of Logistic Regression Model : \n",classification_report(y_pred, y_test))

Accuracy of Logistic Regression Model : 97%
Classification Report of Logistic Regression Model : 
               precision    recall  f1-score   support

         0.0       0.99      0.97      0.98    122989
         1.0       0.54      0.75      0.62      4845

    accuracy                           0.97    127834
   macro avg       0.76      0.86      0.80    127834
weighted avg       0.97      0.97      0.97    127834



In [33]:
from sklearn.model_selection import GridSearchCV
parameter = {"penalty":["l1", "l2", "elasticnet"],
             "C" : [1,2,3,4,5,6,10,20,40,50],
             "max_iter" : [100,200,300,400,500]}
Classfier_Reg = GridSearchCV(log_reg,param_grid=parameter, scoring="accuracy", cv=10)

In [34]:
Classfier_Reg.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 10, 20, 40, 50],
                         'max_iter': [100, 200, 300, 400, 500],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [35]:
print(Classfier_Reg.best_params_)
print(Classfier_Reg.best_score_)
y_pred_tune_grid = Classfier_Reg.predict(X_test)
print("Accuracy is {}%".format(round(accuracy_score(y_pred_tune_grid, y_test)*100)))
print(classification_report(y_pred_tune_grid, y_test))

{'C': 4, 'max_iter': 100, 'penalty': 'l2'}
0.9656038171883672
Accuracy is 97%
              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98    122986
         1.0       0.54      0.75      0.62      4848

    accuracy                           0.97    127834
   macro avg       0.76      0.86      0.80    127834
weighted avg       0.97      0.97      0.97    127834



In [29]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(X_train,y_train)
y_pred_dec = dec_tree.predict(X_test)
accuracy_score(y_pred_dec, y_test)*100

97.06416133423032

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)
accuracy_score(y_pred_knn, y_test)*100

In [30]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
random_forest.fit(X_train,y_train)
y_pred_rfc = random_forest.predict(X_test)
accuracy_score(y_pred_rfc, y_test)*100

97.1189198491794

In [32]:
from sklearn.ensemble import GradientBoostingClassifier
grad_boost = GradientBoostingClassifier()
grad_boost.fit(X_train,y_train)
y_pred_gbc = grad_boost.predict(X_test)
accuracy_score(y_pred_gbc, y_test)*100

97.04304019274997

In [31]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train,y_train)
y_pred_svm = svm.predict(X_test)
accuracy_score(y_pred_svm, y_test)*100

97.16194439663938